# Geoptimaliseerde Energienetwerk Analyse

## 1. Parameterinstellingen

In [76]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- Default parameters ---
default_dwellings = {
    'Vrijstaand_groot':90, 'Vrijstaand_klein':20,
    '2^1_groot':60,        '2^1_klein':50,
    'Rijwoning_groot':80,  'Rijwoning_middel':100,
    'rijwoning_250k':45,   'rijwoning_750':40,
    'rijwoning_750-900':335,
    '3-4k_appartement':80, 'Appartement_2-3k':100,
    '2-3k_sociaal':100,    '2k_sociaal':100
}
default_ev_pct = {
    'Vrijstaand_groot':3.0, 'Vrijstaand_klein':2.0,
    '2^1_groot':2.5,        '2^1_klein':1.8,
    'Rijwoning_groot':1.2,  'Rijwoning_middel':1.0,
    'rijwoning_250k':1.0,   'rijwoning_750':0.5,
    'rijwoning_750-900':0.5,
    '3-4k_appartement':0.5, 'Appartement_2-3k':0.5,
    '2-3k_sociaal':0.5,     '2k_sociaal':0.5
}
default_SF = {
    'daily': 0.60,
    'EV': 0.50,
    'HP': 0.55,
    'PV': 1.00
}
default_pv_scale = 1.0
default_battery_specs = {
    'e_nom': 200000,       # kWh
    'c_rate': 0.25,
    'eff_store': np.sqrt(0.84),
    'eff_disp': np.sqrt(0.84)
}

def calc_p_nom(e_nom_kWh, c_rate):
    return e_nom_kWh * c_rate

# --- Live state ---
dwellings = default_dwellings.copy()
ev_pct = default_ev_pct.copy()
SF = default_SF.copy()
pv_scale = default_pv_scale
battery_specs = default_battery_specs.copy()

# --- Sliders with pretty style ---
def make_int_slider(label, value, minval=0, maxval=500, step=1):
    return widgets.HBox([
        widgets.Label(label, layout=widgets.Layout(width="170px")),
        widgets.IntSlider(value=value, min=minval, max=maxval, step=step,
                          style={'description_width':'0px'},
                          layout=widgets.Layout(width="250px", margin='6px 0 6px 0'))
    ])
def make_float_slider(label, value, minval=0, maxval=10, step=0.1):
    return widgets.HBox([
        widgets.Label(label, layout=widgets.Layout(width="170px")),
        widgets.FloatSlider(value=value, min=minval, max=maxval, step=step,
                            readout_format=".2f", style={'description_width':'0px'},
                            layout=widgets.Layout(width="250px", margin='6px 0 6px 0'))
    ])

# --- 1. Dwellings ---
dw_widgets = [make_int_slider(k, v) for k,v in dwellings.items()]

# --- 2. EV % ---
ev_widgets = [make_float_slider(k, v, 0, 10, 0.1) for k, v in ev_pct.items()]

# --- 3. Scale Factors ---
sf_widgets = [
    make_float_slider("Daily (factor)", SF['daily'], 0, 2, 0.05),
    make_float_slider("EV (factor)", SF['EV'], 0, 2, 0.05),
    make_float_slider("HP (factor)", SF['HP'], 0, 2, 0.05),
    make_float_slider("PV (factor)", SF['PV'], 0, 2, 0.05),
    make_float_slider("PV Scale", pv_scale, 0, 2, 0.05)
]

# --- 4. Battery ---
bat_widgets = [
    make_float_slider('Capacity e_nom (kWh)', battery_specs['e_nom'], 0, 5000, 10),
    make_float_slider('C-rate (1/h)', battery_specs['c_rate'], 0.05, 1, 0.01),
    make_float_slider('Efficiency store', battery_specs['eff_store'], 0, 1, 0.01),
    make_float_slider('Efficiency disp', battery_specs['eff_disp'], 0, 1, 0.01)
]

# --- Tabs Layout ---
tab_contents = [
    widgets.VBox(dw_widgets, layout=widgets.Layout(padding="12px")),
    widgets.VBox(ev_widgets, layout=widgets.Layout(padding="12px")),
    widgets.VBox(sf_widgets, layout=widgets.Layout(padding="12px")),
    widgets.VBox(bat_widgets, layout=widgets.Layout(padding="12px")),
]
tab_titles = ["Dwellings", "EV %", "Scale Factors", "Battery"]

param_tabs = widgets.Tab(children=tab_contents, layout=widgets.Layout(width="450px"))
for i, title in enumerate(tab_titles):
    param_tabs.set_title(i, title)

# --- Battery Suggestion Box (right, updates on apply) ---
suggestion_box = widgets.HTML(
    value='<div style="font-size:15px;padding:10px;border:2px solid #444;border-radius:8px;background:#fafcff;color:#222;width:280px;"><b>Battery suggestion:</b><br><i>Apply parameters to update...</i></div>',
    layout=widgets.Layout(width='320px', min_height='130px')
)

def estimate_battery_and_c_rate():
    # Example: Suggest a battery based on load and PV
    # (replace with your logic for actual min battery!)
    # We'll take "one day of net peak PV excess" as a guess:
    total_load = sum([w.children[1].value for w in dw_widgets])
    total_pv_scale = sf_widgets[4].children[1].value
    total_daily_kWh = total_load * 24 * total_pv_scale * 0.6  # Simple guess
    min_battery = np.round(total_daily_kWh * 0.25, -2) # round to 100
    min_c = np.round(np.clip(total_load/100000, 0.1, 1.0), 2)
    msg = f'''
        <div style="font-size:15px;padding:10px;border:2px solid #147;border-radius:8px;background:#e9f2fc;color:#222;width:280px;">
        <b>🔋 Suggested battery:</b><br>
        <b>Min. capacity (e_nom):</b> <span style="color:#184;">{min_battery:.0f} kWh</span><br>
        <b>Min. C-rate:</b> <span style="color:#184;">{min_c:.2f} 1/h</span><br>
        <span style="color:#555;font-size:13px;">Rule-of-thumb: peak load + 6h avg consumption</span>
        </div>
        '''
    return msg

def apply_params(_):
    global dwellings, ev_pct, SF, battery_specs, pv_scale
    # Dwellings
    for widget, key in zip(dw_widgets, dwellings.keys()):
        dwellings[key] = widget.children[1].value
    # EV pct
    for widget, key in zip(ev_widgets, ev_pct.keys()):
        ev_pct[key] = widget.children[1].value
    # Scale factors
    SF['daily'] = sf_widgets[0].children[1].value
    SF['EV'] = sf_widgets[1].children[1].value
    SF['HP'] = sf_widgets[2].children[1].value
    SF['PV'] = sf_widgets[3].children[1].value
    pv_scale = sf_widgets[4].children[1].value
    # Battery
    battery_specs['e_nom'] = bat_widgets[0].children[1].value
    battery_specs['c_rate'] = bat_widgets[1].children[1].value
    battery_specs['eff_store'] = bat_widgets[2].children[1].value
    battery_specs['eff_disp'] = bat_widgets[3].children[1].value
    battery_specs['p_nom_kW'] = calc_p_nom(battery_specs['e_nom'], battery_specs['c_rate'])
    # Update suggestion
    suggestion_box.value = estimate_battery_and_c_rate()
    print('✔️ Parameters updated!')

def reset_params(_):
    # Dwellings
    for widget, key in zip(dw_widgets, default_dwellings.keys()):
        widget.children[1].value = default_dwellings[key]
    # EV pct
    for widget, key in zip(ev_widgets, default_ev_pct.keys()):
        widget.children[1].value = default_ev_pct[key]
    # Scale factors
    for i, key in enumerate(default_SF.keys()):
        sf_widgets[i].children[1].value = default_SF[key]
    sf_widgets[4].children[1].value = default_pv_scale
    # Battery
    bat_widgets[0].children[1].value = default_battery_specs['e_nom']
    bat_widgets[1].children[1].value = default_battery_specs['c_rate']
    bat_widgets[2].children[1].value = default_battery_specs['eff_store']
    bat_widgets[3].children[1].value = default_battery_specs['eff_disp']
    # Update suggestion
    suggestion_box.value = estimate_battery_and_c_rate()
    print("↩️ All parameters reset to defaults!")

button_apply = widgets.Button(description='Apply Parameters', button_style='success', layout=widgets.Layout(width='140px'))
button_reset = widgets.Button(description='Reset to Default', button_style='warning', layout=widgets.Layout(width='140px'))
button_apply.on_click(apply_params)
button_reset.on_click(reset_params)

# --- Layout: two columns, always visible battery suggestion ---
left_col = widgets.VBox([
    widgets.HTML("<h3 style='color:#184;'>Parameters</h3>"),
    param_tabs
], layout=widgets.Layout(width="460px"))

right_col = widgets.VBox([
    widgets.HTML("<h3 style='color:#147;'"),
    suggestion_box,
    widgets.HBox([button_apply, button_reset], layout=widgets.Layout(justify_content='flex-end', margin='18px 0 0 0'))
], layout=widgets.Layout(width="340px", align_items='flex-end', margin="0 0 0 30px"))

ui_box = widgets.HBox([left_col, right_col], layout=widgets.Layout(align_items='flex-start', margin="0 0 30px 0"))

display(ui_box)

# Trigger initial battery suggestion
suggestion_box.value = estimate_battery_and_c_rate()


> **Tip:** Pas alleen bovenstaande waarden aan en herrun deze cel om alle grafieken en simulaties automatisch bij te werken.

---

In [2]:
def clean_and_index(filepath: Path, freq: str = '15min') -> pd.DataFrame:
    df = pd.read_csv(
        filepath,
        sep=';', decimal=',', encoding='utf-8-sig',
        parse_dates=['Datetime'], dayfirst=False
    )
    df = df.infer_objects(copy=False)
    df = df.ffill().bfill()
    for col in df.columns:
        if col != 'Datetime':
            df[col] = df[col].astype(float)
    return df.set_index('Datetime').asfreq(freq)


def load_all_data(cfg: Config) -> Dict[str, pd.DataFrame]:
    results = Parallel(n_jobs=-1)(
        delayed(clean_and_index)(cfg.paths[atype]) for atype in cfg.types
    )
    return dict(zip(cfg.types, results))

NameError: name 'Path' is not defined

---

In [ ]:
# Data inladen
data_dict = load_all_data(cfg)
archetypes = cfg.types
n = cfg.dwellings

# Profielen berekenen
heating = sum(data_dict[atype]['Heating'] * SF['HP'] * n[atype] for atype in archetypes)
electricity = sum(data_dict[atype]['Elektriciteit'] * SF['daily'] * n[atype] for atype in archetypes)
ev = sum(data_dict[atype]['EV'] * SF['EV'] * (ev_pct[atype]/10) * n[atype] for atype in archetypes)
pv = sum(data_dict[atype]['PV'] * SF['PV'] * pv_scale * n[atype] for atype in archetypes)

# Plot alle componenten in één grafiek
plt.figure(figsize=(12, 5))
heating.plot(label='Heating', linewidth=0.8)
electricity.plot(label='Electricity', linewidth=0.8)
ev.plot(label='EV', linewidth=0.8)
pv.plot(label='PV', linewidth=0.8)
plt.title('Totale Verwarming, Elektriciteit, EV en PV')
plt.xlabel('Tijd')
plt.ylabel('Vermogen (kW)')
plt.legend()
plt.tight_layout()
plt.show()

---

## PyPSA Netwerksimulatie

In [75]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from joblib import Parallel, delayed
from typing import Dict
from dataclasses import dataclass, field

# -- Data/Simulation helpers --
def calc_p_nom(e_nom_kWh, c_rate):
    return e_nom_kWh * c_rate

@dataclass
class Config:
    types: list = field(default_factory=lambda: [
        'Vrijstaand_groot', 'Vrijstaand_klein', '2^1_groot', '2^1_klein',
        'Rijwoning_groot', 'Rijwoning_middel', 'rijwoning_250k', 'rijwoning_750',
        'rijwoning_750-900', '3-4k_appartement', 'Appartement_2-3k',
        '2-3k_sociaal', '2k_sociaal'
    ])
    dwellings: Dict[str, int] = field(default_factory=lambda: dwellings)
    EV_pct:    Dict[str, float] = field(default_factory=lambda: ev_pct)
    SF:        Dict[str, float] = field(default_factory=lambda: SF)
    paths:     Dict[str, Path]  = field(init=False)
    def __post_init__(self):
        self.paths = {atype: Path(f"{atype}.csv") for atype in self.types}

def clean_and_index(filepath: Path, freq: str = '15min') -> pd.DataFrame:
    df = pd.read_csv(filepath, sep=';', decimal=',', encoding='utf-8-sig',
                     parse_dates=['Datetime'])
    df = df.infer_objects(copy=False).ffill().bfill()
    for col in df.columns:
        if col != 'Datetime':
            df[col] = df[col].astype(float)
    return df.set_index('Datetime').asfreq(freq)

def load_all_data(cfg: Config) -> Dict[str, pd.DataFrame]:
    results = Parallel(n_jobs=-1)(
        delayed(clean_and_index)(cfg.paths[atype]) for atype in cfg.types
    )
    return dict(zip(cfg.types, results))

def simulate_battery(load_kw, pv_kw, battery_capacity_kwh, battery_power_kw,
                     eff_charge=1.0, eff_discharge=1.0):
    n = len(load_kw)
    soc = np.zeros(n)
    charge = np.zeros(n)
    discharge = np.zeros(n)
    pv_direct = np.zeros(n)
    pv_curtail = np.zeros(n)
    soc_now = 0.0  # kWh
    for t in range(n):
        demand = load_kw[t]
        pv_gen = pv_kw[t]
        direct = min(pv_gen, demand)
        pv_direct[t] = direct
        rest_pv = max(pv_gen - direct, 0)
        net_demand = demand - direct
        can_charge = min(rest_pv, battery_power_kw, battery_capacity_kwh - soc_now)
        charge[t] = can_charge * eff_charge
        soc_now += can_charge * eff_charge
        can_discharge = min(net_demand, battery_power_kw, soc_now / eff_discharge)
        discharge[t] = can_discharge
        soc_now -= can_discharge * eff_discharge
        pv_curtail[t] = max(rest_pv - can_charge, 0)
        soc_now = min(max(soc_now, 0), battery_capacity_kwh)
        soc[t] = soc_now
    return pd.DataFrame({
        'soc': soc,
        'charge': charge,
        'discharge': discharge,
        'pv_direct': pv_direct,
        'pv_curtail': pv_curtail
    })

def average_week(df, months):
    mask = df.index.month.isin(months)
    df_filtered = df[mask]
    day = df_filtered.index.weekday
    hour = df_filtered.index.hour
    minute = df_filtered.index.minute
    week_profile = df_filtered.groupby([day, hour, minute]).mean()
    week_profile = week_profile.sort_index()
    return week_profile.reset_index(drop=True)

# -- Main function with extra vertical space, all legends outside right, and SUMMER/WINTER label below all plots --
def run_full_avg_weeks():
    cfg = Config()
    data_dict = load_all_data(cfg)
    types = cfg.types
    n = cfg.dwellings

    # Aggregated profiles (kW)
    heating = sum(data_dict[t]['Heating'] * SF['HP'] * n[t] for t in types)
    electricity = sum(data_dict[t]['Elektriciteit'] * SF['daily'] * n[t] for t in types)
    ev_load = sum(data_dict[t]['EV'] * SF['EV'] * (ev_pct[t]/10) * n[t] for t in types)
    pv_gen = sum(data_dict[t]['PV'] * SF['PV'] * pv_scale * n[t] for t in types)

    total_load = (heating + electricity + ev_load)
    idx = total_load.index

    # Months definition
    summer_months = [5,6,7,8,9,10]   # May-Oct
    winter_months = [11,12,1,2,3,4]  # Nov-Apr

    avg_summer = pd.DataFrame({
        'load': average_week(total_load, summer_months).values,
        'pv':   average_week(pv_gen,    summer_months).values,
    })
    avg_winter = pd.DataFrame({
        'load': average_week(total_load, winter_months).values,
        'pv':   average_week(pv_gen,    winter_months).values,
    })

    # Battery simulation
    e_nom = battery_specs['e_nom']
    c_rate = battery_specs['c_rate']
    p_nom = calc_p_nom(e_nom, c_rate)
    eff_store = battery_specs.get('eff_store', 1.0)
    eff_disp = battery_specs.get('eff_disp', 1.0)

    sim_summer = simulate_battery(
        load_kw=avg_summer['load'].values,
        pv_kw=avg_summer['pv'].values,
        battery_capacity_kwh=e_nom,
        battery_power_kw=p_nom,
        eff_charge=eff_store,
        eff_discharge=eff_disp
    )
    sim_winter = simulate_battery(
        load_kw=avg_winter['load'].values,
        pv_kw=avg_winter['pv'].values,
        battery_capacity_kwh=e_nom,
        battery_power_kw=p_nom,
        eff_charge=eff_store,
        eff_discharge=eff_disp
    )

    # Grid import
    import_no_batt_summer = avg_summer['load'] - avg_summer['pv']
    import_with_batt_summer = avg_summer['load'] - sim_summer['pv_direct'] - sim_summer['discharge']
    import_no_batt_winter = avg_winter['load'] - avg_winter['pv']
    import_with_batt_winter = avg_winter['load'] - sim_winter['pv_direct'] - sim_winter['discharge']

    # Peaks
    peak_no_batt = [import_no_batt_summer.max(), import_no_batt_winter.max()]
    peak_with_batt = [import_with_batt_summer.max(), import_with_batt_winter.max()]

    # For plotting
    n_steps = 96 * 7  # 15-min resolution, 7 days
    days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    separator = n_steps
    time = np.arange(n_steps*2)
    xticks = [d*96 for d in range(15)]
    xticklabels = days * 2

    # --- PLOTTING with legends outside right and more space ---
    fig, axes = plt.subplots(4, 1, figsize=(18, 15), sharex=True, gridspec_kw={'hspace': 0.36})

    # 1. SOC
    axes[0].plot(np.concatenate([sim_summer['soc'], sim_winter['soc']]), color='orange', label='SOC')
    axes[0].set_ylabel('kWh')
    axes[0].set_title('Battery SOC – Avg. Summer & Winter Week')
    axes[0].axvline(separator, color='gray', lw=2, linestyle='--')
    axes[0].set_xlim(0, n_steps*2-1)
    axes[0].legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)

    # 2. Charge / Discharge
    axes[1].plot(np.concatenate([sim_summer['charge'], sim_winter['charge']]), color='green', label='Charge')
    axes[1].plot(np.concatenate([sim_summer['discharge'], sim_winter['discharge']]), color='red', label='Discharge')
    axes[1].set_ylabel('kW')
    axes[1].set_title('Battery Charge / Discharge – Avg. Summer & Winter Week')
    axes[1].legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
    axes[1].axvline(separator, color='gray', lw=2, linestyle='--')
    axes[1].set_xlim(0, n_steps*2-1)

    # 3. PV flows
    for i, (sim, label, color, fill1, fill2) in enumerate([
        (sim_summer, "Summer", "#FFD580", 'yellow', 'orange'),
        (sim_winter, "Winter", "#B3D3F5", 'deepskyblue', 'blue')
    ]):
        offset = i * n_steps
        axes[2].plot(np.arange(offset, offset + n_steps), sim['pv_direct'] + sim['charge'], label=f'Total PV {label}', color=color)
        axes[2].fill_between(np.arange(offset, offset + n_steps), 0, sim['pv_direct'], label=None, alpha=0.45, color=fill1)
        axes[2].fill_between(np.arange(offset, offset + n_steps), sim['pv_direct'], sim['pv_direct'] + sim['discharge'], alpha=0.35, label=None, color=fill2)
    axes[2].set_ylabel('kW')
    axes[2].set_title('PV Flows – Avg. Summer & Winter Week')
    axes[2].axvline(separator, color='gray', lw=2, linestyle='--')
    axes[2].set_xlim(0, n_steps*2-1)
    axes[2].legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)

    # 4. Grid import with/without battery + Peak lines
    axes[3].plot(np.concatenate([import_no_batt_summer, import_no_batt_winter]), label='Grid Import (no battery)', color='black', alpha=0.5)
    axes[3].plot(np.concatenate([import_with_batt_summer, import_with_batt_winter]), label='Grid Import (with battery)', color='purple', alpha=0.8)
    axes[3].hlines(peak_no_batt[0], 0, n_steps-1, colors='black', linestyles=':', linewidth=2, label='Peak (no battery)')
    axes[3].hlines(peak_no_batt[1], n_steps, n_steps*2-1, colors='black', linestyles=':')
    axes[3].hlines(peak_with_batt[0], 0, n_steps-1, colors='purple', linestyles='--', linewidth=2, label='Peak (with battery)')
    axes[3].hlines(peak_with_batt[1], n_steps, n_steps*2-1, colors='purple', linestyles='--')
    axes[3].set_ylabel('kW')
    axes[3].set_title('Grid Import (with and without Battery) – Avg. Summer & Winter Week')
    axes[3].legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
    axes[3].axvline(separator, color='gray', lw=2, linestyle='--')
    axes[3].set_xlim(0, n_steps*2-1)

    # -- Axis ticks, day names, night shading --
    for ax in axes:
        ax.set_xticks(xticks)
        ax.set_xticklabels(['']*15)  # Hide here, we add below
        ax.set_xticks([i*24 for i in range(0, n_steps*2//24+1)], minor=True)
        for d in range(14):
            night1 = d*96 + 88  # 22:00–24:00
            night2 = d*96       # 00:00–06:00 next day
            ax.axvspan(night1, night1+8, color='gray', alpha=0.13)
            ax.axvspan(night2, night2+24, color='gray', alpha=0.13)
        ax.axvline(separator, color='gray', lw=2, linestyle='--')
        ax.set_xlim(0, n_steps*2-1)

    # -- Day names under each plot --
    for ax in axes:
        for d in range(14):
            ax.text(d*96+48, ax.get_ylim()[0]-0.06*(ax.get_ylim()[1]-ax.get_ylim()[0]),
                    days[d%7], ha='center', va='top', fontsize=12, color='k')

    # -- SUMMER/WINTER label below all subplots in black --
    fig.subplots_adjust(bottom=0.07, top=0.95, right=0.79)  # Make space for legends on right
    fig.text(0.30, 0.02, 'SUMMER', fontsize=21, color='black', ha='center', va='top', fontweight='bold')
    fig.text(0.63, 0.02, 'WINTER', fontsize=21, color='black', ha='center', va='top', fontweight='bold')

    plt.show()

output = widgets.Output()

def run_simulation_widget(_=None):
    with output:
        output.clear_output(wait=True)
        run_full_avg_weeks()

run_button = widgets.Button(description="Run Simulation", button_style='success')
run_button.on_click(run_simulation_widget)
display(run_button, output)


Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()

In [80]:
!pip install voila